Rand Agent Table with RNN for Time Series: for all agents in a given frame, version 1: doing it for all agents in a scene

#### The way it works: 
* it uses the same multiple parallel series process seen in the rnn notebook for single agents
* but this time, we prepared the dataset table to be done differently
* for ONE GIVEN SCENE, we separate the agent table into their own sub-tables by their track ids, 
* and left outer join them each side by side in a new table called Apis, on the frame-index column
* The rest of Apis are Nan, representing the points in time when a certain track id entity leaves the vision of the ai car.
* We finally apply the same parallel input series process

* The result is bad, we still need to fit random forests some where.

Look at this for more info on multiple parallel series with RNN: https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

In [ ]:
import keras
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
folderpath = "*"
file = folderpath + "\\lyftlong\\rand_agents_table0.csv"
at = pd.read_csv(file)
file = folderpath + "\\lyftlong\\rand_frames_table1.csv"
ft = pd.read_csv(file)
file = folderpath +  "\\lyftlong\\rand_scenes_table1.csv"
st = pd.read_csv(file)

In [ ]:
at = at.merge(ft[['frame_index', 'scene_index']], on='frame_index')

In [ ]:
atcols = ['centroid_x', 'centroid_y', 'extent_x', 'extent_y', 'extent_z',
          'velocity_x', 'velocity_y', 'yaw', 'track_id','frame_index', 'scene_index']

In [ ]:
atrnn = at[atcols]

In [ ]:
scene_indices = pd.unique(at['scene_index'])

In [ ]:
atrnn1 = atrnn[atrnn.scene_index == scene_indices[0]]

In [ ]:
atrnn1 # scene 1

In [ ]:
agent_parallel_input_series = pd.DataFrame()
apis = agent_parallel_input_series

In [ ]:
# PREPARING THE COLUMNS FOR APIS

In [ ]:
atrnn1s = atrnn1 #.sort_values(['track_id', 'frame_index'])

In [ ]:
# making area:
atrnn1s['area'] = atrnn1s['extent_x'] * atrnn1s['extent_y']

In [ ]:
track_ids = pd.unique(atrnn1s['track_id'])
NUM_ENTITIES = len(track_ids)

In [ ]:
apis_cols = ['centroid_x', 'centroid_y', 'area', 'velocity_x', 'velocity_y', 'frame_index']

In [ ]:
apis_frame_indices = pd.unique(atrnn1['frame_index'])

In [ ]:
apis['frame_index'] = apis_frame_indices

In [ ]:
for i in range(len(track_ids)):
    if i%100 == 0:
        print("{0} of {1}".format(i, len(track_ids)))
    ti = track_ids[i]
    # for each track id, get it's subset in atrnn1s
    atrnn1sti = atrnn1s[atrnn1s.track_id == ti]
    # get these columns only
    atrnn1sti = atrnn1sti[apis_cols]
    # rename the columns by labelling it with it's track id
    atrnn1sti = atrnn1sti.add_prefix('ti{0}_'.format(str(i)))
    # undo apis index's renaming for joining to apis
    atrnn1sti.rename(columns = {"ti{0}_frame_index".format(str(i)): "frame_index"},  
                     inplace = True) 
    # LEFT OUTER JOIN with APIS, atrnn1sti
    apis = pd.merge(apis, atrnn1sti, on='frame_index', how='left')

In [ ]:
frame_indexer_col_count = 1
assert apis.shape == (248, 5860 + frame_indexer_col_count)

In [ ]:
def make_partitions(ar: np.array, step: int = 3) -> np.array:
    len_ar = len(ar)
    partitions = np.array([])
    for i in range(len_ar - step):
        X_partition = ar[i: i + step]
        y_partition = ar[i + step]
        partition = dict()
        partition['X'] = ((X_partition))
        partition['y'] = ((y_partition))
        partitions = np.append(partitions, partition)
    return partitions

In [ ]:
apis.drop('frame_index')

In [ ]:
apis = apis.fillna(0)

In [ ]:
atrnn1 = apis

In [ ]:
def make_partitions(ar: np.array, step: int = 3) -> np.array:
    len_ar = len(ar)
    partitions = np.array([])
    for i in range(len_ar - step):
        X_partition = ar[i: i + step]
        y_partition = ar[i + step]
        partition = dict()
        partition['X'] = ((X_partition))
        partition['y'] = ((y_partition))
        partitions = np.append(partitions, partition)
    return partitions

In [ ]:
NSTEPS = 3

In [ ]:
partitions = make_partitions(np.array(atrnn1), NSTEPS)

In [ ]:
TRAIN_PROPORTION = 0.8
TRAIN_PROPORTION_INDEXER = int(len(atrnn1) * 0.8)

In [ ]:
X_train = []
y_train = []
X_test = []
y_test = []
for i in range(len(partitions)): # [:TRAIN_PROPORTION_INDEXER]:
    partition = partitions[i]
    if i <= TRAIN_PROPORTION_INDEXER:
        X_train.append(partition['X'])
        y_train.append(partition['y'])
    else:
        X_test.append(partition['X'])
        y_test.append(partition['y'])        
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
n_steps = NSTEPS
n_features = len(atrnn1.columns)

In [ ]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(X_train, y_train, epochs=300, verbose=0)

In [ ]:
x_inp = X_test[0]
y_targ = y_test[0]

In [ ]:
x_inp = x_inp = x_inp.reshape((1, n_steps, n_features))
x_inp.shape

In [ ]:
y_hat = model.predict(x_inp, verbose=0)

In [ ]:
y_hat # y_pred

In [ ]:
y_targ

In [ ]:
np.sqrt(((y_hat - y_targ)**2)).sum()

In [ ]:
from scipy import spatial

CosineSimilarity = 1 - spatial.distance.cosine(y_hat, y_targ)
CosineSimilarity